## Negative Selection data preparation and file creation

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [2]:
ns_files = pd.read_csv(r'C:\Users\B420615\OneDrive - Standard Bank\Py\SNS_Control.csv')

In [3]:
ns_files = ns_files['Files'].tolist()

In [4]:
path = glob.glob(r'C:\Users\B420615.STANLIB\Neg_Sel\*.csv')

#create two lists of desired column names - one for all columns and one for data columns
adjcols = ['Date','Ticker','Name','BM','AdjfcfY','CECP','TR_1M','Price','FCFY','MCap','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at','seas_1_1an','rec_chg']
data_cols = ['BM','AdjfcfY','CECP','TR_1M','Price','FCFY','MCap','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at','seas_1_1an','rec_chg']
cols_with_commas = ['Price','MCap']
cols_to_avg = ['BM','AdjfcfY','TR_1M','FCFY','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at','seas_1_1an']
cols_to_zero = ['CECP']
excl_cols = ['rec_chg']

In [5]:
for file in ns_files[-1:]:
    try:
        neg_sel_df = pd.read_csv(rf'C:\Users\B420615\OneDrive - Standard Bank\Neg_Sel\{file}.csv', thousands=',')
        #neg_sel_df['Date'] = pd.to_datetime(neg_sel_df['Date'], format='%Y%m%d')


        # clean data so that all values, apart from naming values, and including NaNs, are reflected as float values. Delete unneeded column
        neg_sel_df = neg_sel_df[neg_sel_df['Ticker'].notna()] 
        neg_sel_df.drop(['EPS_ESTIMATE_CHG'], axis=1,inplace=True)
        
        column_mapping = {
            'Date': 'Date',
            'Ticker': 'Ticker',
            'Short Name': 'Name',
            'BENEISH-M': 'BM',
            'AdjFCFYield': 'AdjfcfY',
            'ChgEstimateVSChgPrice': 'CECP',
            'Total Return:M-1': 'TR_1M',
            'FCF Yld:Y': 'FCFY',
            'TR_Momentum': 'TR_12-1',
            'best_eps_chg': 'best_eps_chg',
            'rec_chg_12M': 'rec_chg12M',
            'tp_change': 'tp_chg',
            'fcf_me': 'fcf_me',
            'fcf_at': 'fcf_at',
            'seas_1_1an': 'seas_1_1an',
            'rec_chg': 'rec_chg',
            'Market Cap': 'MCap',
            'Last Px': 'Price'
        }
        
        # Rename columns
        neg_sel_df = neg_sel_df.rename(columns=column_mapping)
        #replace NA's with NaN       
        neg_sel_df.replace('N.A.', np.nan, inplace=True)
        
        def convert_to_float(value):
            if isinstance(value, str):  # Check if the value is a string
                value = value.replace(',', '')  # Remove commas
                try:
                    return float(value)  # Convert to float
                except ValueError:
                    return None  # Return None if conversion fails
            return value  # Return the value unchanged if not a string
        
        neg_sel_df[data_cols] = neg_sel_df[data_cols].map(convert_to_float)
       
        #   replace values with mean value where required 
        for col in cols_to_avg:
            if col in neg_sel_df.columns:
                mean_value = neg_sel_df[col].astype(float).mean()
                neg_sel_df[col] = neg_sel_df[col].fillna(mean_value)

        #   replace values with zero where required
        for col in cols_to_zero:
            if col in neg_sel_df.columns:
                neg_sel_df[col] = neg_sel_df[col].fillna(0.0)

        # calculate z-score for all cols where  mean values are imputed and append to existing data frame
        z_scores = neg_sel_df[cols_to_avg].apply(zscore)
        z_scores.columns = [f'z_{col}' for col in cols_to_avg]
        neg_sel_df = pd.concat([neg_sel_df, z_scores], axis=1)

        # calculate z-score for all cols where  Nan replaced with zero
        z_scores = neg_sel_df[cols_to_zero].apply(zscore)
        z_scores.columns = [f'z_{col}' for col in cols_to_zero]
        neg_sel_df = pd.concat([neg_sel_df, z_scores], axis=1)

        # create a list of excluded stocks
        excl_stocks = []
        for index,row in neg_sel_df.iterrows():
            if pd.isna(row['rec_chg']):
                excl_stocks.append(row['Ticker'])

        # drop na rows where NaN values are in excl_cols
        neg_sel_df = neg_sel_df.dropna(subset = ['rec_chg'])

        #calculate z scores for excl_cols
        z_scores = neg_sel_df[excl_cols].apply(zscore)
        z_scores.columns = [f'z_{col}' for col in excl_cols]

        # add z-score columns to the original DataFrame
        neg_sel_df = pd.concat([neg_sel_df, z_scores], axis=1)

        #calculate z-scores for NS using specified weights
        # Calculate Neg_Sel z-scores using specified weights
        columns = ['z_TR_12-1','z_CECP', 'z_best_eps_chg', 'z_tp_chg', 'z_rec_chg', 'z_fcf_me','z_fcf_at','z_BM']
        neg_sel_df['Neg_Sel'] = neg_sel_df[columns].mean(axis=1)
        
        
        # Divide universe into quintiles
        neg_sel_df['NS_Quintile'] = pd.qcut(neg_sel_df['Neg_Sel'], q=5, labels=False)+1       

        # Post revised and processed stock ranking file
        output_file_path = rf'C:\Users\B420615\OneDrive - Standard Bank\Neg_Sel\Neg_Sel_S{file}.csv'
        neg_sel_df.to_csv(output_file_path, index=False)

    except Exception as e:
        print(f"Error processing file {file}: {e}") 